<a href="https://colab.research.google.com/github/typleasant22/big-data-challenge/blob/main/Music.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 https://cloud.r-project.org/bin/linux/ubuntu bi

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-03-05 17:46:45--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  4.49MB/s    in 0.2s    

2022-03-05 17:46:45 (4.49 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark import SparkFiles

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import IntegerType

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-HW-1").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [11]:
from pyspark import SparkFiles
url ="https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
music_df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Music_v1_00.tsv.gz"), sep = "\t", header = True, inferSchema=True)

# Show DataFrame
music_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   10140119|R3LI5TRP3YIDQL|B00TXH4OLC|     384427924|Whatever's for Us...|           Music|          5|            0|          0|   N|                Y|          Five Stars|Love this CD alon...| 2015-08-31|
|         US|   27664622|R3LGC3EKEG84PX|B00B6QXN6U|     831769051|Same Trailer Diff...|           Music|          5|    

In [13]:
music_reviews_df = music_df.select(["customer_id","review_id", "product_title",
                                         "product_parent", "review_date","review_headline", "review_body"])
music_reviews_df.show()

+-----------+--------------+--------------------+--------------+-----------+--------------------+--------------------+
|customer_id|     review_id|       product_title|product_parent|review_date|     review_headline|         review_body|
+-----------+--------------+--------------------+--------------+-----------+--------------------+--------------------+
|   10140119|R3LI5TRP3YIDQL|Whatever's for Us...|     384427924| 2015-08-31|          Five Stars|Love this CD alon...|
|   27664622|R3LGC3EKEG84PX|Same Trailer Diff...|     831769051| 2015-08-31|A new fave in our...|This is the album...|
|   45946560| R9PYL3OYH55QY| Soaring (Jazz Club)|      14067376| 2015-08-31|          Five Stars|  Excellent / thanks|
|   15146326|R3PWBAWUS4NT0Q|     CARIBBEAN PARTY|     566295619| 2015-08-31|         Three Stars|Nice variety of c...|
|   16794688|R15LYP3O51UU9E|         Pain Killer|     210426072| 2015-08-31|          Five Stars|Purchased as a gi...|
|   32203364|R1AD7L0CC3DSRI|A Thoughtiverse U...

In [14]:
# checking the datatypes of each column
music_reviews_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)



In [16]:
# counting number of rows in apps_customers_df
music_reviews_df.count()

4751577

In [28]:
music_reviews_df= music_reviews_df.groupBy("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
music_reviews_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   51893555|             1|
|   12945148|             5|
|   38897684|            21|
|   15261709|             3|
|   53036614|           125|
|   50509124|             2|
|   47996333|            23|
|   13560803|             3|
|   51680053|            23|
|   52037325|            14|
|   30682568|            58|
|   35871172|            88|
|   52599576|             3|
|   39162192|             6|
|   46859391|             1|
|   22421107|             6|
|   14219983|             4|
|   10944849|             1|
|   45936211|             1|
|    1592147|            23|
+-----------+--------------+
only showing top 20 rows



In [21]:
 #Config settings for RDS

## IMPORTANT : PLEASE ADD YOUR OWN CREDENTIALS FOR RDS ##

mode = "append"
jdbc_url="jdbc:postgresql://homework.cqekbhx09q1f.us-east-1.rds.amazonaws.com"
config = {"user":"username", 
          "password": "Password", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to review_id table in RDS

music_reviews_df.write.jdbc(url=jdbc_url, table="review_id", mode=mode, properties= config)

In [ ]:
# Write dataframe to review_date table in RDS

music_reviews_df.write.jdbc(url=jdbc_url, table="review_date", mode=mode, properties=config)

In [ ]:
# Write dataframe to music_review_table in RDS

music_reviews_df.write.jdbc(url=jdbc_url, table="music_review_table", mode=mode, properties=config)